#Restricted Boltzmann Machines (RBM)

首先需要具备的知识：
- 玻尔兹曼机
- 二部图
- sigmoid函数
- Bayes定理
- 蒙塔卡罗方法
- 马尔科夫链
- MCMC
莫尔兹曼机的详解，百度文库参考：
http://wenku.baidu.com/link?url=6OJeLQmP7XzIUd8ppADSK3JUDI0amdP6FVT4lQo86HwCH_0jFjDnTK4mpfAwe3Do3XResjOCYUNAHFw5J1RaGycPDmxX96pw6GePaG9jAHm

##Energy-Based Models (EBM) 
给予能量的模型就是通过一些属性，计算出一个标量的能量。我们通过学习将能量函数向我们希望的方向改进。例如根据我们的判断希望得到更低的能量。基于能量的模型就是通过能量方程定义一个概率分布模型：
$$p(x)=\frac{e^{-E(x)}}{Z}$$
此时E为能量函数，p为概率模型，我们要让我们的结果计算出的概率最大（可以看出此时能量应该最小），又因为我们要用梯度下降法求解，所以我们可以给p加上负号-p来让-p最小
Z是用来正规化的参数：
$$Z=\sum_x{e^{-E(x)}}$$
一般能量函数可以通过随机梯度下降法优化（：the empirical negative log-likelihood of the training data实验负数log相似度）。

在线性回归中softmax函数是一个例子
$$E(x)= x_{max} - x（1）$$
$$p(x) = \frac{e^{x-x_{max}}}{\sum_{x\in D}{e^{x-x_{max}}}}（2）$$
下面是梯度下降的过程，目的就是让 negativelog-likelihood最小,如下：
$$\iota(\theta,D) = -\frac{1}{N}\sum{\log{p(x,\theta)}}  （3）$$

为了让模型更有表现力，我们引入一些属性，但是这些属性是未知的我们用$h$表示，原本已知的属性还是用x表示：那么P写作：
$$p(x) = \sum_h{P(x,h)}=\sum_h{\frac{e^{-E(x,h)}}{Z}}$$
为了和之前的内容保持一致性，作一下修改：
我们另：$-F(x)$代替$-E(x)$:
$$F(x)=-\log{\sum_h{e^{E(x,h)}}}$$
则：
$$p(x)=\frac{e^{-F(x)}}{Z}$$

$$Z=\sum_x{e^{-F(x)}}$$

#上边给出了概率分布
下面讨论如何让这个含有隐藏属性概率的最大

首先我们的negative log-likelihood gradient如公式（3），这个时候需要通过梯度下降法求解，所以我们计算梯度(尝试推倒一下，很容易得到结果)：
$$-\frac{\partial \log{p(x)}}{\partial \theta} =\frac{\partial F(x)}{\partial \theta}-\sum_x{p(x)\frac{\partial F(x)}{\partial \theta}}$$
可以参考的网址：http://www.cnblogs.com/tornadomeet/archive/2013/03/27/2984725.html

然而这个导数很难求，所以我们用蒙塔卡罗方法（统计模拟方法，使用随机数来解决计算问题）来进行简化。
- 简化第一步：使用期望替代p(X),那么x就是服从p采样规律的
现在的问题就是如何得到期望： Markov Chain Monte Carlo methods 

介绍波尔兹曼机（来源于百度百科）：

玻尔兹曼机：BM是一种对称耦合的随机反馈型二值单元神经网络，由可见层和多个隐层组成，网络节点分为可见单元(visible unit)和隐单元(hidden unit)，用可见单元和隐单元来表达随机网络与随机环境的学习模型，通过权值表达单元之间的相关性

受限玻尔兹曼机：RBM是一种玻尔兹曼机的变体，但限定模型必须为二分图。模型中包含对应输入参数的输入（可见）单元和对应训练结果的隐单元，图中的每条边必须连接一个可见单元和一个隐单元。（与此相对，“无限制”玻尔兹曼机包含隐单元间的边，使之成为递归神经网络。）这一限定使得相比一般玻尔兹曼机更高效的训练算法成为可能，特别是基于梯度的对比分歧（contrastivedivergence）算法

#下面进行RBM的计算描述

首先给出明确的能量函数E
$$E(v,h)$$


In [ ]:
def get_cost_updates(self, lr=0.1, persistent=None, k=1):
        #参数解释，学习速率，
        # compute positive phase
        pre_sigmoid_ph, ph_mean, ph_sample = self.sample_h_given_v(self.input)

        # decide how to initialize persistent chain:
        # for CD, we use the newly generate hidden sample
        # for PCD, we initialize from the old state of the chain
        if persistent is None:
            chain_start = ph_sample
        else:
            chain_start = persistent
        # end-snippet-2
        # perform actual negative phase
        # in order to implement CD-k/PCD-k we need to scan over the
        # function that implements one gibbs step k times.
        # Read Theano tutorial on scan for more information :
        # http://deeplearning.net/software/theano/library/scan.html
        # the scan will return the entire Gibbs chain
        (
            [
                pre_sigmoid_nvs,
                nv_means,
                nv_samples,
                pre_sigmoid_nhs,
                nh_means,
                nh_samples
            ],
            updates
        ) = theano.scan(
            self.gibbs_hvh,
            # the None are place holders, saying that
            # chain_start is the initial state corresponding to the
            # 6th output
            outputs_info=[None, None, None, None, None, chain_start],
            n_steps=k
        )
        # start-snippet-3
        # determine gradients on RBM parameters
        # note that we only need the sample at the end of the chain
        chain_end = nv_samples[-1]

        cost = T.mean(self.free_energy(self.input)) - T.mean(
            self.free_energy(chain_end))
        # We must not compute the gradient through the gibbs sampling
        gparams = T.grad(cost, self.params, consider_constant=[chain_end])
        # end-snippet-3 start-snippet-4
        # constructs the update dictionary
        for gparam, param in zip(gparams, self.params):
            # make sure that the learning rate is of the right dtype
            updates[param] = param - gparam * T.cast(
                lr,
                dtype=theano.config.floatX
            )
        if persistent:
            # Note that this works only if persistent is a shared variable
            updates[persistent] = nh_samples[-1]
            # pseudo-likelihood is a better proxy for PCD
            monitoring_cost = self.get_pseudo_likelihood_cost(updates)
        else:
            # reconstruction cross-entropy is a better proxy for CD
            monitoring_cost = self.get_reconstruction_cost(updates,
                                                           pre_sigmoid_nvs[-1])

        return monitoring_cost, updates
        # end-snippet-4